# 1. Installing dependencies

Here I use these 3 dependencies,
1. Weavite for vector storage and retrieval.
2. Google's generative AI for query generation (could have used open source model like phi-3 as well).
3. Sentence transformers to create vector embeddings for the query sentences.

In [2]:
!pip install weaviate-client==3.26.2
!pip install google-generativeai
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.4/120.4 kB 639.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_

In [18]:
import os
import google.generativeai as genai

os.environ['GOOGLE_API_KEY'] = "API-KEY"
genai.configure(api_key = os.environ['GOOGLE_API_KEY'])

# 2. Pre-processing

For the pre-processing:

1. I import the necessary libraries: pandas for data manipulation and SentenceTransformer for generating sentence embeddings.

2. In the DataProcessor class, I initialize it with paths to a CSV file (containing schema information) and a TXT file (containing question-query pairs). The datasets (schema and question-query pairs) are manually extracted along with a few API calls to a combination of 4 models GPT4, Gemini, LLAMA3 and Claude, where the large language models were asked to generate query-answer pair given the schema of the database. I also load a pre-trained sentence transformer model for vector encoding of questions.

3. My read_data method reads the schema data from the CSV file into a DataFrame. It then reads the TXT file, processes each line into a list of items (question, query, and database ID), and creates another DataFrame.

4. The clean_data method drops any completely empty rows, merges the question-query DataFrame with the schema DataFrame based on the database ID, and cleans the 'db_schema' and 'gemini_mql' columns by removing newlines and extra spaces.

5. In the generate_embeddings method, I use the pre-trained model to generate sentence embeddings for each question and add them as a new column in the DataFrame.

6. The process method ties everything together by calling read_data, clean_data, and generate_embeddings in sequence, then returns the processed DataFrame.

In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer

class DataProcessor:
    def __init__(self, csv_path, txt_path):
        """
        Initialize the DataProcessor with paths to the CSV and TXT files.

        :param csv_path: Path to the CSV file containing schema information.
        :param txt_path: Path to the TXT file containing question-query pairs.
        """
        self.csv_path = csv_path
        self.txt_path = txt_path
        self.schema_append_df = None  # DataFrame to store schema data
        self.df_append = None  # DataFrame to store processed data
        self.model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')  # Load pre-trained sentence transformer model

    def read_data(self):
        """
        Read data from CSV and TXT files, process the TXT data into a structured format.
        """
        # Read schema data from CSV
        self.schema_append_df = pd.read_csv(self.csv_path)

        # Read and process data from TXT
        with open(self.txt_path, 'r') as file:
            data = file.read().split('\n')[3:-1]  # Split by newline, skip header and footer

        # Process each line into a list of items
        rows = [row.split('|') for row in data]

        # Clean each item in the rows
        rows = [[item.strip().replace('```', '').replace('`', '') for item in row if item != ''] for row in rows]

        # Filter rows with more than 3 items (assuming a valid row has 3 items)
        rows = [row for row in rows if len(row) <= 3]

        # Print rows with more than 3 items for debugging
        for row in rows:
            if len(row) > 3:
                print(row[0], '-------', row[1], '------', row[2], '-------', row[3])

        # Create DataFrame from processed rows
        self.df_append = pd.DataFrame(rows, columns=['question', 'gemini_mql', 'db_id'])

    def clean_data(self):
        """
        Clean the data by removing empty rows, merging with schema data, and cleaning string values.
        """
        # Drop rows where all columns are NaN
        self.df_append = self.df_append.dropna(how='all')

        # Merge with schema data based on 'db_id'
        self.df_append = pd.merge(self.schema_append_df, self.df_append, on='db_id')

        # Clean 'db_schema' and 'gemini_mql' columns by removing newlines and extra spaces
        self.df_append['db_schema'] = self.df_append['db_schema'].apply(lambda x: x.replace("\n", "")).apply(lambda x: x.replace("  ", ""))
        self.df_append['gemini_mql'] = self.df_append['gemini_mql'].apply(lambda x: x.replace("\n", "")).apply(lambda x: x.replace("  ", ""))
        self.df_append['gemini_mql'] = self.df_append['gemini_mql'].apply(lambda x: x.replace("```", "")).apply(lambda x: x.replace("  ", ""))

    def generate_embeddings(self):
        """
        Generate sentence embeddings for each question using the pre-trained model.
        """
        self.df_append['vector'] = self.df_append['question'].apply(lambda x: self.model.encode(x))

    def process(self):
        """
        Process the data by reading, cleaning, and generating embeddings.

        :return: Processed DataFrame
        """
        self.read_data()
        self.clean_data()
        self.generate_embeddings()
        return self.df_append

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
csv_path = '/content/drive/MyDrive/weavite/mongodb_array_object.csv'
txt_path = '/content/drive/MyDrive/weavite/mongodb_array_object.txt'

# Create DataProcessor instance and process data
processor = DataProcessor(csv_path, txt_path)
processed_df = processor.process()

# Print the processed DataFrame
processed_df

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


,db_id,db_schema,question,gemini_mql,vector
0,trips,"{""collections"": [{""name"": ""trips"",""indexes"": [...",How many trips started from stations with coor...,"db.trips.find({ ""start_station_location.coordi...","[0.05888682, -0.057749037, -0.026752103, 0.007..."
1,trips,"{""collections"": [{""name"": ""trips"",""indexes"": [...",Can you find trips where the start and end sta...,"db.trips.find({ ""start_station_location"": { $e...","[0.08742453, -0.03934286, 0.015882788, 0.04409..."
2,trips,"{""collections"": [{""name"": ""trips"",""indexes"": [...",What are the average latitude and longitude of...,"db.trips.aggregate([ { $unwind: ""$start_statio...","[0.061616994, -0.07761997, -0.004470903, 0.015..."
3,trips,"{""collections"": [{""name"": ""trips"",""indexes"": [...",How many trips had start stations located with...,"db.trips.find({ ""start_station_location.coordi...","[0.069803596, -0.04402151, -0.03896904, 0.0408..."
4,trips,"{""collections"": [{""name"": ""trips"",""indexes"": [...",Can you find trips where the start station lat...,"db.trips.find({ ""start_station_location.coordi...","[0.09405282, -0.030613618, -0.0054845614, 0.03..."
...,...,...,...,...,...
685,children,"{""collections"": [{""name"": ""children"",""indexes""...",Find the names of students who have at least t...,"db.children.find({favCity: {$size: {$gte: 2}},...","[0.06695643, -0.0020597824, -0.015720407, -0.0..."
686,children,"{""collections"": [{""name"": ""children"",""indexes""...",Retrieve the students who have at least two fr...,"db.children.find({friends: {$size: {$gte: 2}},...","[-0.05059331, 0.03790658, -0.005245966, -0.039..."
687,children,"{""collections"": [{""name"": ""children"",""indexes""...",Find the names of students who have at least t...,"db.children.find({favCity: {$size: {$gte: 3}},...","[0.09914845, 0.026046542, -0.02419481, -0.0088..."
688,children,"{""collections"": [{""name"": ""children"",""indexes""...",Find the names of students who have at least t...,"db.children.find({favCity: {$size: {$gte: 3}},...","[0.10291177, 0.020334102, -0.01746659, -0.0032..."


# 3. Creating Weavite vector store.

For the purpose of utilizing weavite vector store:

I've created a Python class called WeaviateClient that interacts with Weaviate, a vector database and semantic search engine. My implementation uses Weaviate's embedded mode, which means it runs in-memory without needing a separate server. This can be great for development, testing, or small-scale applications where simplicity and quick setup are priorities.

The class defined below has several methods:

1. create_class: I use this to define a schema in Weaviate. It creates a new class (similar to a table in SQL) with specified properties (like columns) if it doesn't already exist. This is crucial for organizing and structuring your data.

2. add_data_object: This method takes a pandas DataFrame and adds each row as a data object in Weaviate. Importantly, it associates each object with a vector (from the 'vector' column). These vectors are key because they represent your data in a high-dimensional space, allowing for semantic similarity searches.

3. get_nearby_objects: This is where the magic happens. Given a query vector, this method finds the most similar objects in the specified class. It returns not just the objects, but also a certainty score indicating how similar they are. This is powerful for tasks like recommendation systems, anomaly detection, or finding semantically similar documents.

**Why use Weaviate?**

It's all about semantic search and working with unstructured data. Traditional databases are great for exact matches, but they struggle with "find me something like this." With vector databases like Weaviate, you can:

* Implement semantic search in text data (find documents with similar meaning, not just keyword matches)
* Work with embeddings from machine learning models (like word embeddings, image features, etc.)
* Build recommendation systems based on content similarity
* Detect anomalies or duplicates by finding "outlier" vectors
* Cluster similar items for data exploration or organization

In my code, I'm setting the foundation for these applications. By storing data with its semantic vectors and providing a way to find similar items, I'm enabling all sorts of smart, meaning-based data operations. It's a step towards making your data not just searchable, but understandable.

In [6]:
import weaviate

class WeaviateClient:
    def __init__(self):
        # Initialize a Weaviate client using the embedded options
        # This means Weaviate will run in-memory without requiring a separate server
        self.client = weaviate.Client(
            embedded_options=weaviate.embedded.EmbeddedOptions(),
        )

    def create_class(self, class_name, properties):
        # Check if the class already exists in the Weaviate schema
        if self.client.schema.exists(class_name):
            print(f"Class {class_name} already exists.")
            return
        else:
            print(f"Creating class {class_name}...")

            # Define the class object with the given name and properties
            class_obj = {
                "class": class_name,
                "properties": properties
            }

            # Create the class in Weaviate schema
            new_class = self.client.schema.create_class(class_obj)

            # Note: The return value 'new_class' is not used (commented out)
            # return new_class

    def add_data_object(self, class_name, df):
        # Extract column names and vectors from the DataFrame
        columns = df.columns.tolist()
        vectors = df['vector'].tolist()

        # Iterate through each row in the DataFrame
        for index, row in df.iterrows():
            # Create a data object dictionary from the row, excluding the 'vector' column
            data_object = {columns[i]: row[columns[i]] for i in range(len(columns)) if columns[i] != 'vector'}

            # Add the data object to the batch, along with its vector
            self.client.batch.add_data_object(data_object, class_name, vector=vectors[index])

        # Create all objects in the batch
        self.client.batch.create_objects()

    def get_nearby_objects(self, class_name, vector, retrieval_columns, limit=10):
        # Define the query vector
        near_vec = {"vector": vector}

        # Build and execute the query:
        # 1. Get objects of the specified class
        # 2. Retrieve specified columns and the certainty score
        # 3. Find objects near the given vector
        # 4. Limit the results
        res = self.client \
            .query.get(class_name, retrieval_columns + ["_additional {certainty}"]) \
            .with_near_vector(near_vec) \
            .with_limit(limit) \
            .do()

        # Return the query results
        return res

In [8]:
class_name = 'MongoDB'
properties = [
        {
          "name": "db_id",
          "dataType": ["text"]
        },
        {
          "name": "db_schema",
          "dataType": ["text"]
        },
        {
          "name": "question",
          "dataType": ["text"]
        },
        {
          "name": "gemini_mql",
          "dataType": ["text"]
        }
      ]

# Initialize WeaviateClient for vector database operations
db_client = WeaviateClient()

# Get Weaviate class configuration from config
class_name = class_name
properties = properties

# Create a class in Weaviate and add processed data
db_client.create_class(class_name, properties)
db_client.add_data_object(class_name, processed_df)

embedded weaviate is already listening on port 8079
Creating class MongoDB...
Embedded weaviate wasn't listening on port 8079, so starting embedded weaviate again
Started /root/.cache/weaviate-embedded: process ID 7397


# 4. Query Generation

For the final part:

I've developed a QueryGeneration class to generate queries based on given inputs. The core of my implementation is a combination of Google's Gemini Pro model for text generation and sentence transformers for encoding and re-ranking. Open source model could have also been used here for query generation, but due to being GPU poor I chose to go with closed source.

My class initializes with a sentence transformer model for encoding questions, a prompt template for query generation, and a cross-encoder model ('BAAI/bge-reranker-base') for re-ranking. I also configure a Gemini Pro model with safety settings to block high-threshold harassment and hate speech.

The main method, **generate_query**, takes inputs like class name, schema, question, and a database. It has a boolean parameter **rag** to toggle Retrieval-Augmented Generation (RAG). When RAG is enabled, I encode the input question, retrieve similar questions from the database, and use the cross-encoder to re-rank these retrieved questions based on their similarity to the input question.

I then take the top two most similar questions as examples and use their associated schemas and queries to fill in my prompt template. This way, the model sees similar examples before generating a query for the current question. When RAG is disabled, I simply fill in the prompt with the given schema and question.

Finally, I use the Gemini model to generate content based on the constructed prompt.

The primary reason for using these technologies is to enhance query generation by leveraging both large language models (Gemini Pro) and information retrieval techniques (RAG). By providing similar examples to the model, I can guide it to generate more accurate and contextually relevant queries.

It's worth noting that the model with RAG generally performs better than without RAG. This is because when RAG is enabled, the model sees examples of similar questions and their corresponding queries. This additional context helps the model understand the expected query structure and the relevant parts of the schema, resulting in more accurate query generation. In contrast, without RAG, the model has to infer the query structure solely from the given schema and question, which can be more challenging, especially for complex or ambiguous questions.


In [20]:
from sentence_transformers import CrossEncoder
import google.generativeai as genai


class QueryGeneration:
    def __init__(self, model):
        # Initialize the QueryGeneration class with a sentence transformer model and a prompt
        self.model = self.get_gemini_model()  # Get the Gemini model
        self.encoding_model = model  # Sentence transformer model for encoding questions
        # self.prompt = prompt  # Prompt template for query generation
        self.cross_encoder = CrossEncoder('BAAI/bge-reranker-base')  # Cross-encoder for re-ranking

    def get_gemini_model(self):
        # Configure and return a Gemini Pro model with safety settings
        safety_settings = [
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_ONLY_HIGH"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_ONLY_HIGH"}
        ]
        model = genai.GenerativeModel('gemini-pro', safety_settings=safety_settings)
        return model

    def generate_query(self, class_name, schema, question, db, prompt, rag=True):
        # Generate a query based on the given class, schema, question, and database
        # rag: boolean to enable/disable Retrieval-Augmented Generation

        if rag:
            # Encode the question using the sentence transformer model
            vector = self.encoding_model.encode(question)

            # Retrieve nearby objects from the database using the question vector
            res = db.get_nearby_objects(class_name, vector, ['db_schema', 'question', 'gemini_mql'], limit=10)
            hits = res["data"]["Get"][class_name]

            # Prepare inputs for cross-encoder (question pairs)
            cross_inp = [[question, hit['question']] for hit in hits]

            # Get cross-encoder scores for re-ranking
            cross_scores = self.cross_encoder.predict(cross_inp)

            # Add cross-encoder scores to hits
            for idx in range(len(cross_scores)):
                hits[idx]['cross-score'] = cross_scores[idx]

            # Sort hits by cross-encoder scores in descending order
            hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)

            # Get the top two hits as examples
            example1 = hits[0]
            example2 = hits[1]

            # Extract schema, question, and query from the examples
            EXAMPLE1_SCHEMA, EXAMPLE1_QUESTION, EXAMPLE1_QUERY = example1['db_schema'], example1['question'], example1['gemini_mql']
            EXAMPLE2_SCHEMA, EXAMPLE2_QUESTION, EXAMPLE2_QUERY = example2['db_schema'], example2['question'], example2['gemini_mql']

            # Replace placeholders in the prompt with actual values
            prompt = prompt.replace("{{SCHEMA}}", schema).replace("{{QUESTION}}", question)
            prompt = prompt.replace("{{EXAMPLE1_SCHEMA}}", EXAMPLE1_SCHEMA).replace("{{EXAMPLE1_QUESTION}}", EXAMPLE1_QUESTION).replace("{{EXAMPLE1_QUERY}}", EXAMPLE1_QUERY)
            prompt = prompt.replace("{{EXAMPLE2_SCHEMA}}", EXAMPLE2_SCHEMA).replace("{{EXAMPLE2_QUESTION}}", EXAMPLE2_QUESTION).replace("{{EXAMPLE2_QUERY}}", EXAMPLE2_QUERY)

        else:
            # If RAG is disabled, just replace schema and question in the prompt
            prompt = prompt.replace("{{SCHEMA}}", schema).replace("{{QUESTION}}", question)

        # Generate content using the Gemini model with the constructed prompt
        return self.model.generate_content(prompt)

#### Both of the below prompts were generated using the Anthropic prompt generator, and were then rigourously tested and improved to work well with the Gemini.

In [21]:
prompt_rag = """I will provide you with the schema for a MongoDB database, along with two examples of natural language questions and their corresponding MongoDB queries. Your task is to convert a new natural language question into a MongoDB query that will retrieve the requested information from the database.

Here is the schema for the MongoDB database:

<schema>
{{SCHEMA}}
</schema>

Here are two examples to help guide you:

<example1>
Schema:
{{EXAMPLE1_SCHEMA}}

Question: {{EXAMPLE1_QUESTION}}

Query:
{{EXAMPLE1_QUERY}}
</example1>

<example2>
Schema:
{{EXAMPLE2_SCHEMA}}

Question: {{EXAMPLE2_QUESTION}}

Query:
{{EXAMPLE2_QUERY}}
</example2>

Now, here is the new question to convert into a MongoDB query:

<question>
{{QUESTION}}
</question>

Think through how to construct the appropriate MongoDB query to answer this question based on the provided schema. Write out your thought process in a <scratchpad>.

Then, provide the final MongoDB query inside <query> tags. The query should be syntactically correct and fully functional to retrieve the requested data from the database.

Remember to only use the fields and structure defined in the provided schema. If the question cannot be answered by the information in the database, say so."""

prompt_nonrag = """I will provide you with the schema for a MongoDB database. Your task is to convert a new natural language question into a MongoDB query that will retrieve the requested information from the database.

Here is the schema for the MongoDB database:

<schema>
{{SCHEMA}}
</schema>

Now, here is the new question to convert into a MongoDB query:

<question>
{{QUESTION}}
</question>

Think through how to construct the appropriate MongoDB query to answer this question based on the provided schema. Write out your thought process in a <scratchpad>.

Then, provide the final MongoDB query inside <query> tags. The query should be syntactically correct and fully functional to retrieve the requested data from the database.

Remember to only use the fields and structure defined in the provided schema. If the question cannot be answered by the information in the database, say so."""

# Schema to perform query on.
posts_schema = '''{"collections": [{"name": "posts","indexes": [{"key": {"_id": 1}},{"key": {"permalink": 1}},{"key": {"author": 1}},{"key": {"title": 1}},{"key": {"tags": 1}},{"key": {"comments.date": 1}}],"uniqueIndexes": [],"document": {"properties": {"_id": {"bsonType": "string"},"body": {"bsonType": "string"},"permalink": {"bsonType": "string"},"author": {"bsonType": "string"},"title": {"bsonType": "string"},"tags": {"bsonType": "array","items": {"bsonType": "string"}},"comments": {"bsonType": "array","items": {"bsonType": "object","properties": {"body": {"bsonType": "string"},"email": {"bsonType": "string"},"author": {"bsonType": "string"},"date": {"bsonType": "date"}}}}}}}],"version": 1}'''

mongodb_querifier = QueryGeneration(processor.model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [22]:
question = 'Find all the "Sci-Fi" related posts written by Chirayu with post length longer than 50 characters'
output = mongodb_querifier.generate_query('MongoDB', posts_schema, question, db_client, prompt = prompt_rag, rag = True)

Embedded weaviate wasn't listening on port 8079, so starting embedded weaviate again
Started /root/.cache/weaviate-embedded: process ID 13703


In [25]:
# Model Generates { $expr: { $gt: [{ $strLenCP: "$body" }, 50] } } which returns the length of the string body using $strLenCP and then compares it to the integer 50.

print(output.text)

<scratchpad>
1. First, we need to filter the posts based on the "tags" field. We want to find all the posts that have "Sci-Fi" in their tags array.
2. Next, we need to filter the posts based on the "author" field. We want to find all the posts written by Chirayu.
3. Finally, we need to filter the posts based on the length of the "body" field. We want to find all the posts where the body length is greater than 50 characters.

So, the final query would be:
```
db.posts.find({
  $and: [
    { tags: "Sci-Fi" },
    { author: "Chirayu" },
    { $expr: { $gt: [{ $strLenCP: "$body" }, 50] } }
  ]
})
```
</scratchpad>

<query>
```
db.posts.find({
  $and: [
    { tags: "Sci-Fi" },
    { author: "Chirayu" },
    { $expr: { $gt: [{ $strLenCP: "$body" }, 50] } }
  ]
})
```
</query>


In [26]:
question = 'Find all the "Sci-Fi" related posts written by Chirayu with post length longer than 50 characters'
output = mongodb_querifier.generate_query('MongoDB', posts_schema, question, db_client, prompt = prompt_nonrag, rag = False)

In [28]:
# Model generates { body: { $gt: 50 } } where body is text and not an integer so this expression is not true.
print(output.text)

<scratchpad>
First, we need to identify posts related to "Sci-Fi" based on the 'tags' field. Since 'tags' is an array, we can use the $in operator to check if 'Sci-Fi' is included in the tags array.

Next, we need to filter posts written by Chirayu based on the 'author' field.

Finally, we need to filter posts with length longer than 50 characters. We can use the $gt operator to compare the length of the 'body' field with 50.
</scratchpad>

<query>
{
  $and: [
    { tags: { $in: ["Sci-Fi"] } },
    { author: "Chirayu" },
    { body: { $gt: 50 } }
  ]
}
</query>


It can be seen comparing the outputs of the 2 requests (with RAG and without RAG) that the LLM with relevant RAG is generating superior result compared to the one with no RAG.

## Why do we need Re-ranking?

Consider the example below.

In [32]:
# Both the first and second query are not good enough to guide the model towards right direction.

near_vec = {"vector": processor.model.encode('''Find posts with comments from the author "Jane Smith" between January 1, 2022 and December 31, 2022''')}
res = db_client.client \
    .query.get("MongoDB", ["db_id", "question","gemini_mql", "_additional {certainty}"]) \
    .with_near_vector(near_vec) \
    .with_limit(10) \
    .do()

hits = res["data"]["Get"]["MongoDB"]
for post in res["data"]["Get"]["MongoDB"]:
        print(post["_additional"]["certainty"], '|', post["db_id"], post["question"],'|', post["gemini_mql"])
        print('---')

0.7474851608276367 | books Find books with the author "Jane Austen" and a page count between 300 and 500 | db.books.find({"authors": "Jane Austen", "pageCount": {$gte: 300, $lte: 500}})
---
0.7126334309577942 | books Find books with the category "History" and published after 2010 | db.books.find({"categories": "History", "publishedDate": {$gt: new Date("2010-01-01")}})
---
0.699367344379425 | customers Find customers with name 'John Doe' or 'Jane Smith' | db.customers.find({"name": {"$in": ["John Doe", "Jane Smith"]}})
---
0.6833274960517883 | trades Find trades with the ticker "FB" and a time range between March 1, 2023 and March 14, 2023. | db.trades.find({ticker: "FB", time: {$gte: ISODate("2023-03-01"), $lt: ISODate("2023-03-15")}})
---
0.6827490627765656 | children Get the students whose date of birth is between January 1, 2010, and December 31, 2012. | db.children.find({dob: {$gte: new Date("2010-01-01"), $lte: new Date("2012-12-31")}}, {first_name: 1, last_name: 1, dob: 1, _id: 

In [33]:
# The second re-ranked query is exactly similar to what we want as an output from the model.

query = '''Find posts with comments from the author "Jane Smith" between January 1, 2022 and December 31, 2022'''
cross_inp = [[query, hit['question']] for hit in hits]
cross_scores = mongodb_querifier.cross_encoder.predict(cross_inp)

for idx in range(len(cross_scores)):
    hits[idx]['cross-score'] = cross_scores[idx]

hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)

for hit in hits:
    print(hit['cross-score'],'|', hit['question'], '|', hit['gemini_mql'])
    print('---')

0.07612825 | Find customers with name 'John Doe' or 'Jane Smith' | db.customers.find({"name": {"$in": ["John Doe", "Jane Smith"]}})
0.020884207 | Get the students whose date of birth is between January 1, 2010, and December 31, 2012. | db.children.find({dob: {$gte: new Date("2010-01-01"), $lte: new Date("2012-12-31")}}, {first_name: 1, last_name: 1, dob: 1, _id: 0})
0.003650334 | Find books with the author "Jane Austen" and a page count between 300 and 500 | db.books.find({"authors": "Jane Austen", "pageCount": {$gte: 300, $lte: 500}})
0.0034265788 | Find books with the category "History" and published after 2010 | db.books.find({"categories": "History", "publishedDate": {$gt: new Date("2010-01-01")}})
0.0015753509 | Find all movies with at least one writer in the "writers" array. | db.movies.find({writers: {$exists: true, $ne: []}})
0.0013581152 | Find restaurants with grade date in February 2022. | db.restaurants.find({ 'grades.date': { $gte: ISODate('2022-02-01'), $lte: ISODate('202

As observed from the above example the re-ranked queries are more relevant to our input question and thus will guide the model towards the right path in generating the MongoDB query.

## Why do we need RAG when we already have few-shot MongoDB query generation?

The need for Retrieval-Augmented Generation (RAG) over few-shot examples for MongoDB query generation stems from the dynamic nature of RAG in providing more contextually relevant and specific examples compared to the static nature of few-shot learning. Here’s a breakdown of why RAG can be more beneficial:

* Fixed Examples: The examples remain the same for every query, which might not always be relevant to the specific query context. This can lead to less accurate or less optimized query generation, as seen in the case of string length calculation above.

* Contextual Relevance: By retrieving and using examples that are directly relevant to the current query, RAG can tailor the prompt to better fit the user's needs, improving the accuracy and quality of the generated MongoDB queries.

* Complex Query Generation: For complex or less common queries, fixed examples may not provide sufficient guidance. RAG can retrieve specialized examples that better match the complexity of such queries.


# 2.1 Build a front-end or presentation for your workflow

I used streamlit for the purpose of front-end, because it is quick to implement.
https://querifier.streamlit.app/

# 2.2 Outline a potential growth pipeline

For this approach:
1. I started by importing pandas and reading a CSV file containing job postings data into a DataFrame called df.
2. I checked for null values in the 'skills_desc' column and then selected the first 20,000 rows, for testing, storing them in df_test. I used only 20,000 rows because of compute limit, but it can be extended to include the complete dataframe.
3. I preprocessed the 'title' and 'description' columns by converting them to lowercase and removing non-alphanumeric characters.
4. I imported the BERTopic library for topic modeling and CountVectorizer from sklearn for text vectorization.
5. To improve the model's performance, I used NLTK to download and include English stopwords in the CountVectorizer.
6. I initialized a BERTopic model with the customized CountVectorizer, setting it to use English language and calculate probabilities.
7. I fit and transformed the model on the combined 'title' and 'description' data, obtaining topics and their probabilities. The model performed better on combination of df_test['title'] and df_test['description'] compared to just the description.
8. I defined a list of search terms related to data science and machine learning, and filtered the topics that contain these terms. Using this I obtained the clusters which contain AI/ML information.
9. Finally, I tested the model with a sample job description for a machine learning role at Samsung Ads, which I found on linkedin, I transformed this example using the model and printed the predicted topic label and its description.

This code demonstrates how I used BERTopic to automatically identify and categorize topics in job postings, with a focus on data science and machine learning roles.

**Reasons for using BERTopic against LDA**

BERTopic is a state-of-the-art topic modeling technique that combines the power of BERT (Bidirectional Encoder Representations from Transformers) with traditional topic modeling methods like TF-IDF and c-TF-IDF (class-based TF-IDF). Here's why it's advantageous for our job posting analysis:

1. Semantics Understanding: BERT is a pre-trained language model that understands the context and semantics of words. Unlike traditional methods like Latent Dirichlet Allocation (LDA) that treat words as independent tokens, BERT captures the nuanced meaning of words based on their context. This is crucial for job postings where terms like "python" could refer to a programming language or a snake, depending on the context.

2. Handling Domain-Specific Language: Job postings often contain industry jargon, technical terms, and acronyms. BERT, being trained on a vast corpus of text, has a good grasp of such domain-specific language. This means it can better understand and group terms like "ML" (machine learning), "AI" (artificial intelligence) into relevant topics.

3. Capturing Phrases: By using CountVectorizer with ngram_range=(1, 2), we allow BERTopic to consider both unigrams and bigrams. This is important because many job skills are expressed as phrases: "machine learning", "data analysis", "project management". Traditional methods might separate these, losing meaning, but BERTopic keeps them together.

4. Improved Topic Coherence: BERTopic uses c-TF-IDF, which considers the frequency of words across topics, not just documents. This leads to more coherent topics. In our case, it helps distinguish between topics like "data science" and "data entry", even though both contain the word "data".

In [2]:
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.10
    Uninstalling Cython-3.0.10:
      Successfully uninstalled Cython-3.0.10


In [1]:
# Read the CSV file containing job postings data
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/weavite/postings.csv')

In [2]:
df.head()

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,expiry,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,1.715990e+12,NaN,NaN,Requirements: \n\nWe are seeking a College or ...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,...,1.715450e+12,NaN,NaN,NaN,1.712858e+12,NaN,0,FULL_TIME,USD,BASE_SALARY
2,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",64896719.0,8.0,NaN,...,1.715870e+12,NaN,NaN,We are currently accepting resumes for FOH - A...,1.713278e+12,NaN,0,FULL_TIME,USD,BASE_SALARY
3,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,YEARLY,"New Hyde Park, NY",766262.0,16.0,NaN,...,1.715488e+12,NaN,NaN,This position requires a baseline understandin...,1.712896e+12,NaN,0,FULL_TIME,USD,BASE_SALARY
4,35982263,NaN,Service Technician,Looking for HVAC service tech with experience ...,80000.0,YEARLY,"Burlington, IA",NaN,3.0,NaN,...,1.716044e+12,NaN,NaN,NaN,1.713452e+12,NaN,0,FULL_TIME,USD,BASE_SALARY


In [3]:
len(df)

123849

In [14]:
# Check for null values in the columns
df['skills_desc'].isnull().sum(), df['title'].isna().sum(), df['description'].isna().sum()

(121410, 0, 7)

In [5]:
# Select the first 20,000 rows, due to compute limit.
df_test = df.iloc[:20000]

# Preprocess 'title' and 'description' columns:
# Convert to lowercase and remove non-alphanumeric characters
df_test['title'] = df_test['title'].str.lower().str.replace('[^\w\s]', '')
df_test['description'] = df_test['description'].str.lower().str.replace('[^\w\s]', '')

<ipython-input-5-a90e85815c54>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['title'] = df_test['title'].str.lower().str.replace('[^\w\s]', '')
<ipython-input-5-a90e85815c54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['description'] = df_test['description'].str.lower().str.replace('[^\w\s]', '')


In [6]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Add stopwords to improve model performance
stopwords = list(stopwords.words('english')) #+ freq['Representation'][0]

# Initialize CountVectorizer with bigrams and stopwords
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=stopwords)


# Initialize BERTopic model with customized CountVectorizer
model = BERTopic(
    vectorizer_model=vectorizer_model,
    language='english', calculate_probabilities=True,

)

# Check available methods in the BERTopic model
dir(model)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_auto_reduce_topics',
 '_c_tf_idf',
 '_cluster_embeddings',
 '_combine_zeroshot_topics',
 '_create_topic_vectors',
 '_extract_embeddings',
 '_extract_representative_docs',
 '_extract_topics',
 '_extract_words_per_topic',
 '_get_param_names',
 '_guided_topic_modeling',
 '_images_to_text',
 '_is_zeroshot',
 '_map_predictions',
 '_map_probabilities',
 '_merged_topics',
 '_outliers',
 '_preprocess_text',
 '_reduce_dimensionality',
 '_reduce_to_n_topics',
 '_reduce_topics',
 '_save_representative_docs',
 '_sort_mappings_by_frequency',
 '_top_n_idx_sparse',
 '_top_n_values_sparse',
 '_update_topic_size',
 '_zeroshot_topic_mode

In [7]:
# Fit and transform the model on combined 'title' and 'description' data
topics, probs = model.fit_transform(df_test['title']+' '+df_test['description'])

# Get topic information
freq = model.get_topic_info()

# Print top 10 topics
print(freq.head(10))

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


   Topic  Count                                               Name  \
0     -1   5632                     -1_work_experience_team_skills   
1      0    418             0_clinical_research_scientific_medical   
2      1    214                  1_data_analyst_analytics_business   
3      2    208  2_accounting_accountant_financial_senior accou...   
4      3    199                      3_galt_earnings_owners_credit   
5      4    196  4_administrative_office_administrative assista...   
6      5    153              5_security_cybersecurity_cyber_threat   
7      6    150  6_account_account manager_sales_account executive   
8      7    143  7_practical nurse_licensed practical_nursing_p...   
9      8    138                 8_hardware_support_desktop_windows   

                                      Representation  \
0  [work, experience, team, skills, required, pos...   
1  [clinical, research, scientific, medical, regu...   
2  [data, analyst, analytics, business, business ...   
3  [a

In [8]:
# Define search terms related to data science and ML and filter topics that contain the search terms to identify the AI/ML clusters.
search_terms = ['data', 'science', 'machine', 'learning', 'artificial', 'intelligence']
idx = []
for ind, row in freq.iterrows():
  for i in search_terms:
      if i in row['Name'] or i in row['Representation']:
          idx.append(ind)

In [12]:
# Example job description for testing
example = ["""Machine Learning Model Engineer Samsung Ads is an advanced advertising technology company in rapid growth that focuses on enabling brands to connect with Samsung TV audiences as they are exposed to digital media by using the industry’s most comprehensive data to build the world’s smartest advertising platform. Being part of an international company such as Samsung and doing business around the world means that we get to work on the most challenging projects with stakeholders and teams located around the globe.
We are proud to have built a world-class organization grounded in an entrepreneurial and collaborative spirit. Working at Samsung Ads offers one of the best environments in the industry to learn just how fast you can grow, how much you can achieve, and how good you can be. We thrive on problem-solving, breaking new ground, and enjoying every part of the journey. Machine learning lies in the core of the advertising industry. This is no exception to Samsung Ads. At Samsung Ads, we are actively exploring the latest machine learning techniques to improve our existing systems and products and create new revenue streams. As a machine learning model engineer of the Samsung Ads Platform Intelligence (PI) team, you will have access to unique Samsung proprietary data to develop and deploy a wide spectrum of large-scale machine learning products with real-world impact. You will work closely with and be supported by a talented engineering team and top-notch researchers to work on exciting machine learning projects and state-of-the-art technologies. You will be welcomed by a unique learning culture and creative work atmosphere. This is an exciting and unique opportunity to get deeply involved in envisioning, designing and implementing cutting-edge machine learning products with a growing team."""]

In [13]:
# Transform the example job description using the trained model
res = model.transform(example)
res[0]

[48]

In [15]:
# Print the predicted topic label and its description
print("The Predicted label for example is ", res[0][0], model.topic_labels_[res[0][0]])

The Predicted label for example is  48 48_ai_machine learning_machine_ml
